In [1]:
import numpy as np
import cv2 as cv
import glob

### Breakdown of  the code 

1. It sets up the termination criteria for the corner subpixel refinement process using **cv.TERM_CRITERIA_EPS** and **cv.TERM_CRITERIA_MAX_ITER**.
2. It defines the object points (**objp**) for the chessboard pattern, which are the (**x, y**) coordinates of the chessboard corners in the real-world space.
3. It initializes empty lists to store the object points (**objpoints**) and image points (**imgpoints**).
4. It loops through all the images in the Images folder with the .jpg extension.
5. For each image, it converts it to grayscale and tries to find the chessboard corners using **cv.findChessboardCorners**.
6. If the chessboard corners are found, it appends the object points (**objp**) to objpoints and the refined corner points (**corners2**) to imgpoints.
7. It also draws the detected chessboard corners on the image and displays it using **cv.drawChessboardCorners** and **cv.imshow**.
8. After processing all the images, it calls **cv.calibrateCamera** to calculate the camera matrix (**mtx**), distortion coefficients (**dist**), rotation vectors (**rvecs**), and translation vectors (**tvecs**).
9. It reads an image (23.jpg) and uses **cv.getOptimalNewCameraMatrix** to calculate the optimal new camera matrix (**newcameramtx**) and the region of interest (**roi**) for undistortion.
10. It undistorts the image using **cv.undistort** with the calculated camera matrix, distortion coefficients, and the optimal new camera matrix.
11. It crops the undistorted image using the calculated region of interest.
12. Finally, it saves the undistorted and cropped image as calibresult.png using **cv.imwrite**.

### Termination criteria

Here, criteria defines the termination criteria for iterative algorithms used in OpenCV functions. It combines **cv.TERM_CRITERIA_EPS** (termination criteria based on a certain accuracy threshold) and **cv.TERM_CRITERIA_MAX_ITER** (termination criteria based on a maximum number of iterations). The values 30 and 0.001 specify the maximum number of iterations allowed (30 iterations) and the desired accuracy (0.001), respectively

In [2]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

### Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)

In [3]:
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

### Arrays to store object points and image points from all the images.

In [4]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [42]:
images = glob.glob('Images/*.jpeg')

In [43]:
for fname in images:
 img = cv.imread(fname)
 gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
 # Find the chess board corners
 ret, corners = cv.findChessboardCorners(gray, (6,9), None)
 
 # If found, add object points, image points (after refining them)
 if ret == True:
   objpoints.append(objp)
 
   corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
   imgpoints.append(corners2)
 
 # Draw and display the corners
 cv.drawChessboardCorners(img, (6,9), corners2, ret)
 cv.imshow('img', img)
 cv.waitKey(10000)
 
cv.destroyAllWindows()

### Calibration
Now that we have our object points and image points, we are ready to go for calibration. We can use the function, cv.calibrateCamera() which returns the camera matrix, distortion coefficients, rotation and translation vectors etc

In [44]:
img = cv.imread(fname)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

### Undistortion

Now, we can take an image and undistort it. OpenCV comes with two methods for doing this. However first, we can refine the camera matrix based on a free scaling parameter using cv.getOptimalNewCameraMatrix(). If the scaling parameter alpha=0, it returns undistorted image with minimum unwanted pixels. So it may even remove some pixels at image corners. If alpha=1, all pixels are retained with some extra black images. This function also returns an image ROI which can be used to crop the result.

In [45]:
img = cv.imread('26.jpeg')
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [46]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True

In [47]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst2 = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
 
# crop the image
x, y, w, h = roi
dst2 = dst[y:y+h, x:x+w]
cv.imwrite('calibresult2.png', dst)

True

In [48]:
mean_error = 0
for i in range(len(objpoints)):
 imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
 error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
 mean_error += error
 
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 44.31421428356387
